In [2]:
import pandas as pd
import numpy as np 

import matplotlib.pyplot as plt

In [3]:
data = pd.read_csv('MyData.csv')

In [4]:
data.head()

,Unnamed: 0,Recording_ID,Genus,Specific_epithet,Subspecies,English_name,Recordist,Country,Locality,Latitude,Longitude,Vocalization_type,Audio_file,License,Url,Quality,Time,Date
0,1,441880,Parus,major,NaN,Great Tit,Gerda van Deelen,Netherlands,"Amersfoort, Amersfoort, Utrecht",52.1468,5.3788,alarm call,//www.xeno-canto.org/441880/download,//creativecommons.org/licenses/by-nc-sa/4.0/,https://www.xeno-canto.org/441880,no score,09:00,2018-11-10
1,2,440223,Parus,major,corsus,Great Tit,Nicolas Martinez,France,"Algajola, Haute-Corse, Corse",42.6080,8.8610,call,//www.xeno-canto.org/440223/download,//creativecommons.org/licenses/by-nc-sa/4.0/,https://www.xeno-canto.org/440223,no score,09:30,2018-10-03
2,3,440205,Parus,major,NaN,Great Tit,Frank Roos,Netherlands,"Maarssen, Maarssen, Utrecht",52.1331,5.0489,begging call,//www.xeno-canto.org/440205/download,//creativecommons.org/licenses/by-nc-sa/4.0/,https://www.xeno-canto.org/440205,no score,14:00,2018-06-09
3,4,439558,Parus,major,NaN,Great Tit,Alain Verneau,France,"Théville, Manche, Normandie",49.6682,-1.4283,song,//www.xeno-canto.org/439558/download,//creativecommons.org/licenses/by-nc-sa/4.0/,https://www.xeno-canto.org/439558,no score,18:00,2018-06-04
4,5,439316,Parus,major,NaN,Great Tit,Nicolas Martinez,France,"Algajola, Haute-Corse, Corse",42.6080,8.8610,call,//www.xeno-canto.org/439316/download,//creativecommons.org/licenses/by-nc-sa/4.0/,https://www.xeno-canto.org/439316,no score,11:00,2018-10-06
